In [ ]:
import numpy
import pylo

%matplotlib widget
import matplotlib.pyplot

In [ ]:
halo = 1
Lx, Ly = 100., 100.

domain = pylo.Domain(1, 50, 1, 51, 1, 10, halo)

domain.T.c1[:] = numpy.linspace(-Lx/2, Lx/2, domain.T.c1.shape[0])
domain.T.c2[:] = numpy.linspace(-Ly/2, Ly/2, domain.T.c2.shape[0])
domain.T.H[1:-1, 1:-1] = 1.
domain.T.mask[1:-1, 1:-1] = 1

domain.initialize()
domain.T.mask[...] = 0
domain.T.mask[2:-2, 2:-2] = 1

In [ ]:
period = 600
omega = 2*numpy.pi/period
cfl = 1.
umax=omega*Lx/2
dt_cfl=cfl*min(Lx/(domain.T.c1.shape[0]+1), Ly/(domain.T.c2.shape[0]+1)) / umax
no_of_revolutions = 5
Nmax=no_of_revolutions*round(2*numpy.pi/omega/dt_cfl)
tmax=no_of_revolutions*2*numpy.pi/omega
timestep = tmax/Nmax

In [ ]:
u, u_ = domain.array(fill=0.)
v, v_ = domain.array(fill=0.)
var, var_ = domain.array(fill=1.)
u[:, :] = -omega * domain.U.c2[:, numpy.newaxis]
v[:, :] = omega * domain.V.c1[numpy.newaxis, :]
u[domain.U.mask==0] = 0.
v[domain.V.mask==0] = 0.
var[int(0.2*domain.shape[1]):int(0.4*domain.shape[1]),int(0.2*domain.shape[2]):int(0.4*domain.shape[2])] = 5.

In [ ]:
fig = matplotlib.pyplot.figure()
ax = fig.gca()
pc = ax.quiver(u,v)

In [ ]:
matplotlib.pyplot.ioff()
fig = matplotlib.pyplot.figure()
ax = fig.gca()
pc = ax.pcolormesh(var_)
cb = fig.colorbar(pc)
fig.show()

In [ ]:
adv = pylo.Advection(domain, scheme=1)
for i in range(1, Nmax):
    adv.calculate(u_, v_, timestep, var_)
    pc.set_array(var_.ravel())
    fig.canvas.draw()